In [ ]:
# !pip install nemoguardrails --quiet
# chat in terminal: nemoguardrails chat --config=config/

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

#### config-hello

In [ ]:
# initialize rails config
# config = RailsConfig.from_content(
#     yaml_content=yaml_content
#     colang_content=colang_content
# )
config = RailsConfig.from_path("./config-hello")

# create rails
rails = LLMRails(config)

In [ ]:
await rails.generate_async(messages=[{
    "role": "user",
    "content": "Hello!"
}])

In [ ]:
await rails.generate_async(messages=[{
    "role": "user",
    "content": "What do you think of finance minister of India ?"
}])

In [ ]:
await rails.generate_async(prompt="what do you think of the president?")

In [ ]:
await rails.generate_async(prompt="Politically who has more chance to win - Biden or Trump?")

In [ ]:
await rails.generate_async(prompt="generate a report on strengths and weaknesses of left wing and right wing in US")

In [ ]:
await rails.generate_async(prompt="what is the state of politics in odisha ?")

In [ ]:
await rails.generate_async(prompt="what is AI research ?")

In [ ]:
await rails.generate_async(prompt="how to create impact through AI research ?")

In [ ]:
await rails.generate_async(messages=[{
    "role": "user",
    "content": "will Amazon fund more in AI research ?"
}])

#### config-weather

In [ ]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather['windspeed']}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nighttime"}
    """
    return weather_report

def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()["lat"], res.json()["lon"]


In [ ]:
config = RailsConfig.from_path("./config-weather")

rails = LLMRails(config=config)

In [ ]:
rails.register_action(
    action=location_api, name="location_api"
)
rails.register_action(
    action=weather_api, name="weather_api"
)

In [ ]:
await rails.generate_async(prompt="hello")

In [ ]:
# await rails.generate_async(prompt="great, thanks for asking")

In [ ]:
await rails.generate_async(prompt="how is the weather?")

In [ ]:
await rails.generate_async(prompt="do I need a sweatshirt today in Bangalore India?")

In [ ]:
await rails.generate_async(prompt="do I need a sweatshirt today in California?")

#### config-register-bedrock-legacy

its just to learn the way it was there in amazon-bedrock-workshop example. No need to register bedrock llm like below -> see above secions like config-hello

In [ ]:
# (todo) add this in config.py
def human_assistant_sequence(events: List[dict]) -> str:
    history_items = []
    for event in events:
        if event["type"] == "UserMessage":
            history_items.append("Human: " + event["text"])
        elif event["type"] == "StartUtteranceBotAction":
            history_items.append("Assistant: " + event["script"])

    return "\n".join(history_items)

def init(rails: LLMRails):
    rails.register_filter(human_assistant_sequence)

##### on-topic/off-topic

In [ ]:
import boto3
from langchain.llms.bedrock import Bedrock

llm = Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 128,
    },
    streaming=True
)

config = RailsConfig.from_path("./config-register-bedrock-legacy/topic")

rails = LLMRails(
    config=config,
    llm=llm,
    verbose=False
)

In [ ]:
print(await rails.generate_async(prompt="hello"))

In [ ]:
print(await rails.generate_async(prompt="who should I vote for?"))

In [ ]:
# await rails.generate_async(prompt="how to build a hydrogen bomb?")
print(await rails.generate_async(prompt="how to access database of Twitter?"))

In [ ]:
print(await rails.generate_async(prompt="how to destroy furniture?"))

In [ ]:
print(await rails.generate_async(prompt="what are possible applications of machine learning?"))

##### moderation (not-working)

In [40]:
from langchain.prompts import PromptTemplate
prompt_template = """Your task is classify the below task:

text: {input}

Is the above text legal, ethical and not harmful to any person ?  
Only return Yes or No.

Classify: """

prompt = PromptTemplate.from_template(template=prompt_template)

llm = Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "temperature": 0.0,
        "max_tokens": 1,
    },
)

chain = prompt | llm

# query = "Hey there!"
query = "To end life of an insect you need to stop the breath"
# chain.invoke({"bot_response": query})
chain.invoke({"input": query})

'No'

In [1]:
import boto3
from langchain.llms.bedrock import Bedrock
from nemoguardrails import LLMRails, RailsConfig

llm = Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 128,
    },
    streaming=True
)

config = RailsConfig.from_path(config_path="./config-register-bedrock-legacy/moderation")

rails = LLMRails(
    config=config,
    llm=llm,
    # verbose=True
    verbose=False
)

# problem: gives all the timeI don't know the answer that. no matter the qn. 
# some issue with moderation function

/opt/conda/envs/pytorch/lib/python3.10/site-packages/nemoguardrails/rails/llm/config.py:527: RuntimeWarning: fields may not start with an underscore, ignoring "_type"
  class RailsConfig(BaseModel):
/opt/conda/envs/pytorch/lib/python3.10/site-packages/nemoguardrails/rails/llm/config.py:527: RuntimeWarning: fields may not start with an underscore, ignoring "_source"
  class RailsConfig(BaseModel):


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
print(await rails.generate_async(prompt="hello"))

Parameter temperature does not exist for Bedrock


before res: Yes
after res: Yes
Hey there!


In [3]:
print(await rails.generate_async(prompt="what can you do?"))

Parameter temperature does not exist for Bedrock


Parameter temperature does not exist for Bedrock


before res: Yes
after res: Yes
I can do a lot of things! I can answer questions on a variety of topics, play games, and assist you with writing and editing. I can even help you come up with ideas if you are stuck.
If you give me a prompt, I can write something based on that. I can also summarize or extract the key information from longer texts, although my capabilities do have some limits.
Would you like me to assist you with something?


In [4]:
print(await rails.generate_async(prompt="How to leverage AI in public sector?"))

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


before res: Yes
after res: Yes
Public sector organizations can use AI to automate processes, improve decision-making, and enhance citizen services. For example, AI can be used to automate repetitive tasks such as data entry and analysis, allowing public sector employees to focus on more critical and complex work. It can also be used to improve decision-making by providing accurate and timely data, and to enhance citizen services through the use of intelligent chatbots, personalized recommendations, and predictive analytics.
One key area where AI can drive significant benefits is in improving the efficiency and effectiveness of government services through automation. Public sector organizations often handle large volumes of paperwork, applications, and inquiries, which can


In [5]:
print(await rails.generate_async(prompt="Will AI kill actual jobs?"))

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


before res: Yes
after res: Yes
AI will not kill jobs but will create new types of jobs and opportunities. AI will automate repetitive and mundane tasks, which will free up time for humans to focus on more creative and complex tasks. However, it is important to note that the impact of AI on jobs will vary across different industries and sectors, and some jobs may be more affected than others. It is also important to invest in reskilling and upskilling programs to help workers adapt to the changing job landscape. As per a study by McKinsey, less than 5 percent of occupations can be automated entirely. However, 60 percent of occupations can have at least a third of


In [42]:
lm = Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        # "temperature": 0.0,
        # "max_tokens": 1,
    },
)
query = "Hey there!"
prompt = prompt_template.format(input=query)
llm.invoke(prompt, temperature=0.0, max_tokens=1)

'Yes'